Импорты библиотек

In [2]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
 
from keras.optimizers import RMSprop

import random
import time
import re

2025-05-19 13:47:32.413069: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-19 13:47:32.462519: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-19 13:47:32.811910: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-19 13:47:34.863063: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747651655.285657    2242 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747651655.36

Обработка текста

In [7]:
def prepare_text(text):
     text = re.sub(r'[^\w\s]|[\n\r]+', '', text)
     return text.lower()

def pretty_print(text):
    count = 0
    for word in text.split():
        print(word, " ", end="")
        if count % 7 == 0:
            print(" ")
        count += 1

with open('input.txt', 'r') as file:
    text = file.read()

text = prepare_text(text)
print("Число слов в тексте: ", len(text.split()))
pretty_print(text)

# Получаем алфавит
vocabulary = sorted(list(set(text)))
 
# Создаем словари, содержащие индекс символа и связываем их с символами
char_to_indices = dict((c, i) for i, c in enumerate(vocabulary))
indices_to_char = dict((i, c) for i, c in enumerate(vocabulary))

# Разбиваем текст на цепочки длины max_length
# Каждый временной шаг будет загружать очередную цепочку в сеть
max_length = 5
steps = 1
sentences = []
next_chars = []

# Создаем список цепочек и список символов, которые следуют за цепочками
for i in range(0, len(text) - max_length, steps):
    sentences.append(text[i: i + max_length])
    next_chars.append(text[i + max_length])

Число слов в тексте:  16409
однажды   
вечером  я  сидел  во  дворе  возле  песка   
и  ждал  маму  она  наверно  задерживалась  в   
институте  или  в  магазине  или  может  быть   
долго  стояла  на  автобусной  остановке  не  знаю   
только  все  родители  нашего  двора  уже  пришли   
и  все  ребята  пошли  с  ними  по   
домам  и  уже  наверно  пили  чай  с   
бубликами  и  брынзой  а  моей  мамы  все   
еще  не  былои  вот  уже  стали  зажигаться   
в  окнах  огоньки  и  радио  заиграло  музыку   
и  в  небе  задвигались  темные  облака  они   
были  похожи  на  бородатых  старикови  мне  захотелось   
есть  а  мамы  все  не  было  и   
я  подумал  что  если  бы  я  знал   
что  моя  мама  хочет  есть  и  ждет   
меня  гдето  на  краю  света  я  бы   
моментально  к  ней  побежал  а  не  опаздывал   
бы  и  не  заставлял  ее  сидеть  на   
песке  и  скучатьи  в  это  время  во   
двор  вышел  мишка  он  сказал  здоровои  я   
сказал  здоровомишка  сел  со  мной  и  взял   
в  рук

Создание датасета

In [8]:
# Создаем тренировочный набор
# Создаем битовые вектора для входных значений
# (Номер_цепочки-Номер_символа_в цепочке-Код_символа)
X = np.zeros((len(sentences), max_length, len(vocabulary)), dtype = np.bool)
# Выходные данные
# (Номер_цепочки-Код_символа)
y = np.zeros((len(sentences), len(vocabulary)), dtype = np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_to_indices[char]] = 1
    y[i, char_to_indices[next_chars[i]]] = 1

Коллбэк для отрисовки

In [9]:
class LossPlotter(tf.keras.callbacks.Callback):
    def __init__(self, plot_frequency=1):
        super(LossPlotter, self).__init__()
        self.plot_frequency = plot_frequency
        self.losses = []
        self.epochs = []
        self.start_time = None

    def on_train_begin(self, logs=None):
        self.losses = []
        self.epochs = []

    def on_epoch_begin(self, epoch, logs=None):
        self.start_time = time.time()

    def on_epoch_end(self, epoch, logs=None):
        self.epochs.append(epoch + 1)
        self.losses.append(logs.get('loss'))
        epoch_time = time.time() - self.start_time 

        if (epoch + 1) % self.plot_frequency == 0:
            self.plot_loss()

    def plot_loss(self):
        plt.clf() 
        plt.plot(self.epochs, self.losses, 'b', label='Training loss')
        plt.title(f'Training Loss (Epoch {self.epochs[-1]})')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.pause(0.1)
        plt.show(block=False)

Создание модели

In [10]:
model = Sequential()
model.add(LSTM(128, input_shape =(max_length, len(vocabulary))))
model.add(Dense(len(vocabulary)))
model.add(Activation('softmax'))
optimizer = RMSprop(learning_rate = 0.01)
model.compile(loss ='categorical_crossentropy', optimizer = optimizer)

E0000 00:00:1747651973.213784    2242 cuda_executor.cc:1228] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1747651973.214366    2242 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/home/romka/source/mod-lab07-lstm/.venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Обучение

In [11]:
plotter = LossPlotter()
model.fit(X, y, batch_size = 128, verbose=1, epochs = 50)

Epoch 1/50


2025-05-19 13:53:00.294548: E tensorflow/core/util/util.cc:131] oneDNN supports DT_BOOL only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.


740/740 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - loss: 2.5572
Epoch 2/50
740/740 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - loss: 1.9781
Epoch 3/50
740/740 ━━━━━━━━━━━━━━━━━━━━ 10s 13ms/step - loss: 1.7936
Epoch 4/50
740/740 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - loss: 1.6776
Epoch 5/50
740/740 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - loss: 1.5999
Epoch 6/50
740/740 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - loss: 1.5400
Epoch 7/50
740/740 ━━━━━━━━━━━━━━━━━━━━ 10s 13ms/step - loss: 1.4893
Epoch 8/50
740/740 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - loss: 1.4569
Epoch 9/50
740/740 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - loss: 1.4292
Epoch 10/50
740/740 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - loss: 1.3952
Epoch 11/50
740/740 ━━━━━━━━━━━━━━━━━━━━ 10s 13ms/step - loss: 1.3702
Epoch 12/50
740/740 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - loss: 1.3504
Epoch 13/50
740/740 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - loss: 1.3325
Epoch 14/50
740/740 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - loss: 1.3172
Epoch 15/50
740/740 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/ste

Гененерация

In [12]:
def sample_index(preds, temperature = 1000.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def generate_text(length, diversity):
    # Случайное начало
    start_index = random.randint(0, len(text) - max_length - 1)
    generated = ''
    sentence = text[start_index: start_index + max_length]
    generated += sentence
    for i in range(length):
            x_pred = np.zeros((1, max_length, len(vocabulary)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_to_indices[char]] = 1.
 
            preds = model.predict(x_pred, verbose = 0)[0]
            next_index = sample_index(preds, diversity)
            next_char = indices_to_char[next_index]
 
            generated += next_char
            sentence = sentence[1:] + next_char
    return generated

generated = generate_text(1500, 0.2)


In [14]:
print(len(generated.split()))
pretty_print(generated)



287
азал   
а  у  меня  в  сторонам  и  все   
равно  я  все  время  побежал  к  ним   
стал  разбудить  с  нами  сокращенные  пропускали  на   
не  стал  думал  как  будто  я  все   
время  корчился  по  стулу  изо  все  стал   
думал  как  стрела  что  не  ответил  что   
он  подошел  на  меня  на  два  раз   
мне  с  нами  сокращенные  разными  как  столу   
потому  что  не  понял  как  будто  она   
вся  красные  на  своей  тогда  я  быстро   
получается  покататься  в  комнаты  но  как  будто   
я  все  думал  что  он  подошел  на   
меня  в  место  и  все  время  побрыжить   
на  меня  почему  наверно  потом  потом  потом   
подошел  к  мороженщина  стал  думал  что  она   
все  стал  разбудить  старый  мор  подушку  на   
свете  и  побежал  когда  потом  в  сторонам   
и  все  время  колесо  и  колот  я   
кричал  с  нами  стрела  что  я  все   
думать  то  мне  в  голосом  успокаего  все   
время  побежал  к  столу  и  стал  разбудить   
и  вот  он  скажуи  выйчил  побольшую  к